## Gaussian Mixture Models

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [8]:
mu1 = -1
mu2 = 3
sig1 = 0.5
sig2 = 1
N = 100
np.random.seed(10)
x11=np.random.randn(N,1)*sig1 + mu1
x12=np.random.randn(N,1)*sig1 + mu1+3
x21=np.random.randn(N,1)*sig2 + mu2
x22=np.random.randn(N,1)*sig2 + mu2+3
c = np.vstack((np.zeros((N,1)), np.ones((N,1))))
x1 = np.hstack((x11,x12))
x2 = np.hstack((x21,x22))

X = np.hstack( (np.vstack( (x1,x2) ),c) )
np.random.shuffle(X)
dataset = pd.DataFrame(data=X, columns=['x','y','c'])

In [9]:
dataset

,x,y,c
0,-0.334207,2.058738,0.0
1,3.264395,5.636814,1.0
2,-0.669884,1.856185,0.0
3,1.995284,4.484924,1.0
4,3.588041,4.739172,1.0
...,...,...,...
195,-1.060953,1.742545,0.0
196,-1.365985,2.723583,0.0
197,-1.133659,1.885027,0.0
198,3.602022,5.744720,1.0


In [ ]:
class K_Means(object):
    """K Means Algorithm."""
    def __init__(self, M, iters = 250):
        self. M = M
        self.iters = iters
               
    def initial_centroids(self):
        random_idx = np.random.permutation(self.X.shape[0])
        selected_index= random_idx[0:self.M]
        return self.X[selected_index]
    
    def compute_dist(self):
        cluster_dist = np.zeros((self.X.shape[0], self.M))
        for i in range(self.M):
            cluster_dist[:,i]= np.sqrt(np.sum((self.X - self.centroids[i])**2, axis=1))
        return cluster_dist
                    
    def update_centroids(self):
        for i in range(self.M):
            self.centroids[i] = np.mean(self.X[self.clusters == i])       
        return 
        
        
    def fit(self, X):
        self.clusters = np.zeros(X.shape[0])
        self.X= X
        self.centroids= self.initial_centroids()
        
        for i in range(self.iters):
            self.clusters = np.argmin(self.compute_dist(), axis =1)
            self.update_centroids()
              
    def clusters(self):
        return self.clusters
    
    def centroids(self):
        return self.centroids
    

In [ ]:
class GaussianMixtureModel:
    def __init__(self,
                 num_clusters: int = 1,
                 tolerance: float = 1e-5,
                 num_iters: int = 10,
                 ):
        self.num_clusters = num_clusters
        self.tolerance: tolerance
        self.num_iters = num_iters
        self.X = None
        
    def initial_means(self):
        n = self.X.shape[0]
        random_idx = np.random.permutation(n)
        selected_index = random_idx[0:self.num_clusters]
        return self.X[selected_index]
    
    def compute_mean_cov(self):
        n = self.X.shape[0]
        Xmean = np.mean(X, axis=0)
        Xcentred = X - Xmean
        Xcov = (1/n) * (Xcentred.T@Xcentred)
        return Xmean, Xcov
    
    def initialize_phis(num_clusters):
        ### Set all component distribution prior estimates 
        to the uniform distribution
        ###
        return [1/num_clusters for _ in range(num_clusters)]
    
    def initialize_cluster():
        n = self.X.shape[0]
        random_idx = np.random.uniform(0, self.num_clusters, n)
        return random_idx
    
    def pXiClusti(row: int, cluster: int):
        
        
    def fit(self, X: pd.DataFrame):
        n, d = X.shape
        self.X = X.copy()
        # Initialize mean and cov of clusters
        self.cov = np.ones(self.num_clusters, d, d)
        self.means = self.initial_means()
        self.Xmean, self.Xcov = self.compute_mean_cov()
        for i in range(self.num_clusters):
            self.cov[i] = self.Xcov
        
        # Initialize clusters
        self.clusters = self.initialize_cluster()
        self.phis = self.initialize_phis(self.num_clusters)
        
        iter = 0
        self.w = np.zeros((n, self.num_clusters))
        while (iter < self.num_iters) or (mean_norm >= self.tolerance):
            # Perform Expectation step(E step)
            for cluster in range(self.num_clusters):
                for row in range(n):
                    self.w[row, cluster] = self.phis[cluster] 
                        * self.pXiClusti(row, 
                                         cluster)
            iter += 1

        
        